<a href="https://colab.research.google.com/github/AjayPatsariya/project-1-/blob/master/project2_Distract_Driver_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import os
import cv2
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.applications import MobileNet # This is a pre-trained model available in tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input # Every pre-trained model has its own preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [3]:
from skimage.color import rgb2gray
from skimage import io
from skimage import color
from PIL import Image
import PIL.Image

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
os.getcwd()

'/content'

In [5]:
dilist = pd.read_csv('/content/drive/MyDrive/Distractdriver/driver_imgs_list.csv', na_values='na')
dilist.head()

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [6]:
 base_model = MobileNet(weights = 'imagenet', include_top = False)

17225924/17225924 [==============================] - 0s 0us/step


In [7]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv1 (Conv2D)              (None, None, None, 32)    864       
                                                                 
 conv1_bn (BatchNormalizati  (None, None, None, 32)    128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, None, None, 32)    0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, None, None, 32)    288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, None, None, 

In [8]:
CLASSES = 10
base_model_output = base_model.output # Will result in 7x7x1024 values [Last layer in base_model => conv_pw_13 (Conv2D) : (None, 7, 7, 1024)]
global_avg_layer = GlobalAveragePooling2D(name = 'avg_pool')(base_model_output)
dense_layer = Dense(128, activation = 'relu')(global_avg_layer)
predictions = Dense(CLASSES, activation = 'softmax')(dense_layer)
model = Model(inputs = base_model.input, outputs = predictions)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv1 (Conv2D)              (None, None, None, 32)    864       
                                                                 
 conv1_bn (BatchNormalizati  (None, None, None, 32)    128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, None, None, 32)    0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, None, None, 32)    288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, None, None, 32)    128   

In [10]:
for layer in base_model.layers:
    layer.trainable = False

In [14]:
train_image = []
image_label = []


for i in range(10):
    print('now we are in the folder C',i)
    imgs = os.listdir("/content/drive/MyDrive/Distractdriver/images/train"+str(i))
    for j in range(len(imgs)):
    #for j in range(100):
        img_name = "/content/drive/MyDrive/Distractdriver/images/train"+str(i)+"/"+imgs[j]
        img = cv2.imread(img_name)
        #img = color.rgb2gray(img)
        img = img[50:,120:-50]
        img = cv2.resize(img,(224,224))
        label = i
        driver = dilist[dilist['img'] == imgs[j]]['subject'].values[0]
        train_image.append([img,label,driver])
        image_label.append(i)

now we are in the folder C 0


FileNotFoundError: ignored

In [13]:

train_image = []
image_label = []

for i in range(10):
    print('now we are in the folder C', i)
    img_dir = "/content/drive/MyDrive/Distractdriver/images/train" + str(i)

    # Check if the directory exists
    if os.path.exists(img_dir):
        imgs = os.listdir(img_dir)
        for j in range(len(imgs)):
            img_name = os.path.join(img_dir, imgs[j])
            img = cv2.imread(img_name)

            if img is not None:
                img = img[50:, 120:-50]
                img = cv2.resize(img, (224, 224))
                label = i


                driver = dilist[dilist['img'] == imgs[j]]['subject'].values[0]

                train_image.append([img, label, driver])
                image_label.append(i)
            else:
                print(f"Failed to load image: {img_name}")
    else:
        print(f"Directory does not exist: {img_dir}")


print(len(train_image), len(image_label))


now we are in the folder C 0
Directory does not exist: /content/drive/MyDrive/Distractdriver/images/train0
now we are in the folder C 1
Directory does not exist: /content/drive/MyDrive/Distractdriver/images/train1
now we are in the folder C 2
Directory does not exist: /content/drive/MyDrive/Distractdriver/images/train2
now we are in the folder C 3
Directory does not exist: /content/drive/MyDrive/Distractdriver/images/train3
now we are in the folder C 4
Directory does not exist: /content/drive/MyDrive/Distractdriver/images/train4
now we are in the folder C 5
Directory does not exist: /content/drive/MyDrive/Distractdriver/images/train5
now we are in the folder C 6
Directory does not exist: /content/drive/MyDrive/Distractdriver/images/train6
now we are in the folder C 7
Directory does not exist: /content/drive/MyDrive/Distractdriver/images/train7
now we are in the folder C 8
Directory does not exist: /content/drive/MyDrive/Distractdriver/images/train8
now we are in the folder C 9
Director